# Reconstructing the land-cover of prehistoric landscapes

The purpose of this notebook is to :
1. Load pollen abundance time-series data extracted from the European Pollen Database for a selection of sites I am studying in the development of my PhD thesis.
2. Explore, consider the limitations of, and clean that data.
3. Support the systematic assignment of pollen types identified in the empirical data to the categorical land-cover types which will be represented in my simulation models. This is a form of modelling in itself, and serves as an abstraction couched in terms of the notion of a plant functional type. That is, plant _species_ which are postulated to be functionally identical as far as the model is concerned are assigned to the same plant functional group. This will be achieved using regular expressions to embelish the data in a pandas dataframe.
4. Apply the Landscape Reconstruction Algorithm (LRA) to the pollen abundance data to infer the _proportion_ of landscape occupied from each plant functional group.
5. Produce, for each of my empirical study sites, time-series of the proportion of landscape occupied for each of the functional groups represented in the model for the duration of time for which there is abundance data for each study site. This will be presented in the form of a `.csv` file and a plot for each study site. 

The only input required to run this notebook is a path to the file `site_pollen_abundance_ts.csv` which is output from [`epd-query`](https://github.com/lanecodes/epd-query).

In [ ]:
from pathlib import Path
import os
import sys
import re
from typing import List

import unidecode

import matplotlib as mpl
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

mpl.rcParams['font.family'] = 'CMU Sans Serif'
import matplotlib.pyplot as plt
%matplotlib inline
#import seaborn as sns
from ipywidgets import interact, fixed

import pandas as pd
import numpy as np

In [ ]:
TMP_DIR = Path('../tmp')
OUTPUT_DIR = Path('../outputs')

# 1. Load pollen data from file

In [ ]:
epd_data = pd.read_csv(TMP_DIR / 'site_pollen_abundance_ts.csv')

In [ ]:
epd_data.head()

In [ ]:
epd_data.groupby(['sitename', 'e_']).size()

## 2. Explore, condider the limitations of, and clean pollen core data

### Check numbers of samples in each core, narrow core selection

Load data for one of the three Navarres cores

In [ ]:
nav_dat = epd_data[epd_data['e_'] == 469]

In [ ]:
nav_dat.sort_values(by='count', ascending=False)

We see that much of the pollen recorded in the database for this sediment core corresponds to [pollen spike](https://quantpalaeo.wordpress.com/2017/07/28/pollen-spikes/), with `varcode` values of `conc.spk`. 

In [ ]:
nav_dat['sample_'].unique()

More troublingly, navares core 469, NAVA1 has only 15 samples.

In [ ]:
nav2 = epd_data[epd_data['e_'] == 470]
nav2['sample_'].unique()

In [ ]:
nav3 =  epd_data[epd_data['e_'] == 471]
nav3['sample_'].unique()

Going forward, I'll prefer NAVA3 over NAVA1 and NAVA2 since it contains more samples. If I find something which makes NAVA3 seem unreliable, I may reconsider. For now, drop NAVA1 and NAVA2 from the `epd_data` dataframe.

In [ ]:
epd_data = epd_data[~epd_data['e_'].isin([469, 470])]

### Look at top ten pollen contributing species for each study site, remove sediment spike

In [ ]:
def print_top_species(epd_data):
    for ssite in epd_data['sitename'].unique():
        print('\n'+ssite)
        df = epd_data[epd_data['sitename']==ssite]
        df = df.groupby(['var_', 'varcode', 'varname']).agg({'count' : 'sum'})
        print(df.sort_values(by='count', ascending=False).head(5))
    del df

print_top_species(epd_data)

Navarres alone seems to have a lot of pollen spike in it. Also Monte Areo mire and Charco da Candieira have Lycopodium spike added. See [here](https://palynology.wordpress.com/2012/10/07/pollen-spike/) for background on pollen spike. To keep analyses between sites consistent, I will exclude these. 

In [ ]:
def remove_varcodes(df: pd.DataFrame, varcodes: List[str]) -> pd.DataFrame:
    """Remove rows corresponding to specified varcodes from epd DF."""
    return df[~df['varcode'].isin(varcodes)]


def remove_varcodes_test_df():
    return pd.DataFrame({
        'varcode': ['goodvar1', 'badvar1', 'badvar2', 'goodvar2'],
        'count': np.random.randint(0, 4000, size=4)
    })


def test_remove_varcodes(test_df):
    res_df = remove_varcodes(test_df, ['badvar1', 'badvar2'])
    assert res_df.iloc[0]['varcode'] == 'goodvar1'
    assert res_df.iloc[1]['varcode'] == 'goodvar2'
    assert len(res_df.index) == 2
    
test_remove_varcodes(remove_varcodes_test_df())   

In [ ]:
exclude_pollen_spike = True
if exclude_pollen_spike:
    epd_data = remove_varcodes(
        epd_data, ['Spi/tab', 'Lyc(ad)', 'Lyc(ct)', 'Lyc']
    )

Also note that San Rafael has a significant proportion of Botryococcus in its samples. This is a type of green algae. Since this doesn't correspond to any _land_ plant species, we exclude it

In [ ]:
aquatic_plant_codes = [
    'Bry',
    'Zyg-T',
    'Spr-T',
    'Pot',      # Potamogeton, aquatic plant
    'Clo.i-T',  # Closterium idiosporum, green algae
    'Spi.cf.s', # Spirogyra cf. scrobiculata, green algae
    'Trl.s',    # Trilete spore(s),  not from modern terrestrial plant
]

exclude_non_land_plants = True
if exclude_non_land_plants:
    epd_data = remove_varcodes(epd_data, aquatic_plant_codes)

Identified lots of moss (Sphagnum) in, e.g. Atxuri. Exclude this

In [ ]:
exclude_mosses = True
if exclude_mosses:
    epd_data = remove_varcodes(epd_data, ['Sph'])

Fungal spores such as Glomus turn up in Navarres. Exclude

In [ ]:
fungal_species_codes = [
    'Glomus',
    'Pos',  # Polyadosporites, fungal spore http://www.redalyc.org/html/454/45437346003/index.html
]

exclude_fungi = True
if exclude_fungi:
    epd_data = remove_varcodes(epd_data, fungal_species_codes)

Remove records corresponding to pollen which could not be identified

In [ ]:
unrecognised_species_codes = [
    'Ind.unkn',  # found in navarres
    'T16C',
]

In [ ]:
exclude_unrecognised = True
if exclude_unrecognised:
    epd_data = remove_varcodes(epd_data, unrecognised_species_codes)

At this point, `epd_data` contains entries for all:
1. sediment cores
2. samples (depths/ ages)
3. species (careful to exclude pollen spike)

In [ ]:
epd_data

#### Give each site an easily typed `sitecode` to refer to as an index

It will be convenient to be able to refer to sites as an index. To make these easy to type, create a `sitecode` column which strips out spaces and removes any unicode names

In [ ]:
print(epd_data['sitename'].unique())

In [ ]:
epd_data['sitecode'] = (
    epd_data['sitename']
    .apply(unidecode.unidecode)
    .str.replace(' ', '_')
    .str.lower()
)

print(epd_data.sitecode.unique())

In [ ]:
epd_data.head()

#### Drop unnecessary columns

I plan to use `sitename` as an index going forward because it's natural to think in terms of study sites. This means I don't need other information in the dataframe I take forward in my analyses at the study site level of detail. So this information can easily be rerieved if needs be when debugging, I save this to disk and remove the extra columns

In [ ]:
site_meta_fields = ['sitecode', 'sitename', 'site_', 'sigle', 'e_', 'chron_']
site_meta = epd_data.groupby(site_meta_fields).size().rename('num_records')
site_meta.to_csv(TMP_DIR / 'site_metadata.csv', encoding='utf8', header=True)
epd_data = epd_data.drop(
    [x for x in site_meta_fields if x != 'sitecode'], axis=1
)
epd_data

`sample_` (a database key from the EPD) is also redundant at this point, since we can idenify each sample from its `agebp`. Similarly each variable (pollen species) is uniquely identified by its `varcode` so we can also drop `var_`.

In [ ]:
epd_data = epd_data.drop(['sample_', 'var_'], axis=1)

In [ ]:
epd_data.head()

Check `agebp` and `count` can be converted to `int` without loss of data, and do the conversion

In [ ]:
def convert_field_to_int(df: pd.DataFrame, field: str) -> pd.DataFrame:
    """Convert named float field to int if no data would be lost."""
    assert (~df[field].isna()).all(), f'missing data found in {field}'
    assert ((df[field] - df[field].astype(int)) == 0).all(), (
     f'casting {field} to int caused loss of data'
    )
    df[field] = df[field].astype(int)
    return df

In [ ]:
epd_data = (
    epd_data
    .pipe(lambda df: convert_field_to_int(df, 'agebp'))
    .pipe(lambda df: convert_field_to_int(df, 'count'))
)

In [ ]:
epd_data

Create a unique index

In [ ]:
epd_data = epd_data.set_index(['sitecode', 'agebp', 'varcode']).sort_index()

Find index is unexpectedly not unique

In [ ]:
epd_data.index.is_unique

Check which sites duplicates are coming from

In [ ]:
epd_data[epd_data.index.duplicated()].groupby(level=['sitecode']).count()

In [ ]:
pct_affected = 107 / len(epd_data.loc['charco_da_candieira'].index) * 100
print(f'{round(pct_affected, 2):.2f}% of charco_da_candieira entries are duplicates')

As less than 1% of Charco da Candieira sample/ species combinations are affected, we will simply assume that where multiple entries are associated for a species in a single sample, the correct count is obtained by summing any duplicates. No other site's data are affected by this issue.

In [ ]:
initial_index_len = len(epd_data.index)

In [ ]:
varcode_to_varname_df = (
    epd_data.reset_index(level='varcode')[['varcode', 'varname']]
    .drop_duplicates()
    .set_index('varcode')
)

In [ ]:
epd_data = (
    epd_data['count']
    .groupby(level=['sitecode', 'agebp', 'varcode']).sum()
    .to_frame()
    .join(varcode_to_varname_df)
)

In [ ]:
assert initial_index_len - len(epd_data.index) == 107
assert epd_data.index.is_unique

Rename `count` to avoid an understandable but irritating namespace collision with the `pd.Series.count` method.

In [ ]:
epd_data = epd_data.rename(columns={'count': 'pcount'})

In [ ]:
epd_data

In [ ]:
epd_data.loc['navarres'].head()

In [ ]:
epd_data.groupby(level=['sitecode', 'agebp']).sum()

`epd_data` is now prepped and ready to use for subsequent analyses. Serialise a csv file so it can be retrieved without rerunning the above cells.

In [ ]:
epd_data.to_csv(TMP_DIR / 'clean_epd_data.csv')

## 3. TODO Relate identified pollen species with model-dependent plant functional types

In [ ]:
try:
    epd_data
except NameError:
    epd_data = (
        pd.read_csv(TMP_DIR / 'clean_epd_data.csv')
        .set_index(['sitecode', 'agebp', 'varcode'])
    )   

### Identify land-cover types with pollen species

Retrieve a list of unique `varname`-s found amongst the sediment cores analysed thus far in the notebook.

In [ ]:
unique_species = epd_data.reset_index()[['varname', 'varcode']].drop_duplicates()
unique_species.head()

~~Aim is to construct a list of dicts with keys `pattern` and `lct`. Intended to mean: if _this_ pattern is found in a species name, map it to _this_ land cover type. ~~

Can't map individual species to individual land cover type. E.g. pine exists in both pine and transition forest. How to attribute 

how to distinguish a small area of pine forest's contribution from a large area of transition forest? They might generate similar amounts of pollen, but result from very different landscapes.

Maybe I need to further simplify the land cover types I'd like to represent?

#### Map land land cover types to species

Landcover types which are represented in my model are as follows:

##### 1. Water/ Quarry
No pollen produced

##### 2. Burnt
No pollen produced

#### 3. Barley 
Grass pollen produced, belongs to the Poaceae (formerly known as Gramineae) family.

None present at the start of a simulation

#### 4. Wheat 
Grass pollen produced, belongs to the Poaceae (formerly known as Gramineae) family

None present at the start of a simulation

#### 5. Depleated Agricultural Land
No pollen produced

None present at the start of a simulation

#### 6. Shrubland
- Grasses (Poaceae, formerly Gramineae, family)
- Juniper (Genus:	Juniperus, belongs to cypress family Cupressaceae)

#### 7. Pine forest
Anything belonging to the pinus genera

#### 9. Deciduous forest
- Beech family, Fagaceae
- Chestnut (Castanea genus)

#### 10. Oak forest
- Anything in quercus

##### Comment on Transition Forest
- Originally in model
- No way to distinguish from pollen between pine forest, oak forest and transition forest. As it stands, it makes sense to integrate out the transition forest state, so we create the possibility of transitioning directly between pine and oak, subject to the kind of environmental conditions which would support transition forest.

In [ ]:
pol_lct_maps = {    
    # SHRUBLAND
    # grasses
    r'(.*(?i)Poaceae.*|.*(?i)Gramineae.*|.*(?i)Cerealia.*)' : 'shrubland',
    # juniper
    r'.*(?i)Juniperus.*' : 'shrubland',
    # and cypress family
    r'.*(?i)Cupressaceae.*' : 'shrubland',
    # quillwort (prolific in Sanabria Marsh)
    r'.*(?i)Isoetes.*' : 'shrubland',
    # Goosefoot family (prolific in e.g. San Rafael)
    r'.*(?i)Chenopodiaceae.*' : 'shrubland',
    # Mugwort genus (prolific in e.g. San Rafael)
    r'.*(?i)Artemisia.*' : 'shrubland',
    # flowering plants in the same family as lettuce, dendelions etc
    r'.*(?i)Cichorioideae.*' : 'shrubland',
    # family of shrubby plants
    r'.*(?i)Asteroideae.*' : 'shrubland',    
    # sedge family (superficially resemble grasses)
    r'.*(?i)Cyperaceae.*' : 'shrubland', # see e.g. Atxuri    
    # heather
    r'.*(?i)Calluna vulgaris.*' : 'shrubland', 
    # heather family
    r'(.*(?i)Ericaceae.*|.*(?i)Erica-type.*|.*(?i)Erica arborea-type.*)' : 'shrubland', 
    #  celery, carrot, parsley family
    r'.*(?i)Umbelliferae.*' : 'shrubland', 
    # celery and marthwort genus
    r'.*(?i)Apium.*' : 'shrubland', 
    # box plant (shrubby tree)
    r'.*(?i)Buxus.*' : 'shrubland',
    # genus of flowering plants, buttercup genus
    r'.*(?i)Ranunculus.*' : 'shrubland',
    # doc/ sorrel genus
    r'.*(?i)Rumex.*' : 'shrubland',
     # bracken/ ferns. Associated with pine forest??
    r'(.*(?i)Pteridium.*|.*(?i)Polypodium.*|.*(?i)Filicales.*)' : 'shrubland', 
    # genus of gymnosperm shrubs
    r'.*(?i)Ephedra.*' : 'shrubland',    
    # flowering plants found in wet regions
    r'(.*(?i)Sparganium.*|.*(?i)Typha angustifolia.*)' : 'shrubland', 
    # plantain/ fleawort genus
    r'.*(?i)Plantago.*' : 'shrubland',
    # olive genus
    r'.*(?i)Olea.*' : 'shrubland',
    
    # PINE FOREST
    r'\s*(?i)Pinus\s*' : 'pine_forest',
    
    # DECIDUOUS FOREST
    # chestnut
    r'.*(?i)Castanea.*' : 'deciduous_forest',
    # birch
    r'.*(?i)Betula.*' : 'deciduous_forest',
    # Beech family
    r'.*(?i)Fagaceae.*' : 'deciduous_forest',
    # Beech genus
    r'.*(?i)Fagus.*' : 'deciduous_forest',
    # Alder genus
    r'.*(?i)Alnus.*' : 'deciduous_forest',
    # Hazel
    r'.*(?i)Corylus.*' : 'deciduous_forest',
    # Willow
    r'.*(?i)Salix.*' : 'deciduous_forest',  
    # Hornbeam
    r'.*(?i)Carpinus.*' : 'deciduous_forest',  
    
    # OAK FOREST
    r'.*(?i)Quercus.*' : 'oak_forest'
}

Define a function which, given a species name, returns a list of land cover types. h

In [ ]:
def get_lct(species_name, pol_lct_dict, verbose=False):
    """Given a species name, map it to a land cover type.
    
    Throw a ValueError if species name matches more than one land cover type.
    """
    lcts = []
    for r in pol_lct_dict.keys():
        if re.match(r, species_name):
            lcts.append(pol_lct_dict[r])
            if verbose:
                print(r + ' matches ' + species_name)
    
    if len(lcts) > 1:
        # len(lcts)
        raise ValueError('Species name {0} matched multiple land cover type '\
                         'regex strings: {1}'.format(species_name, lcts))
    elif len(lcts) == 0:
        return None
    
    else:
        return lcts[0]

print(get_lct('Carpinus', pol_lct_maps))

Apply `get_lct` to each species included in the chronology

In [ ]:
unique_species['lct'] = unique_species.varname.apply(lambda x: get_lct(x, pol_lct_maps))

In [ ]:
mapped_species = unique_species[unique_species['lct'].notnull()]
mapped_species.to_csv(TMP_DIR / 'species_to_landcover_mapping.csv', index=False)

#### Evaluate proportion of pollen, for each study site, accounted for by land-cover type mapping

For each study site, find the overall percentage of pollen contributed by each species

In [ ]:
all_pollen_by_site_species = epd_data.groupby([epd_data.index.get_level_values(0), 'varcode']).sum().reset_index(1)
all_pollen_by_site = all_pollen_by_site_species.groupby(level=0).sum().rename(columns={'pcount':'site_total'})
all_pollen_by_site_species = all_pollen_by_site_species.join(all_pollen_by_site)
all_pollen_by_site = None # remove temporary dataframe used in join
all_pollen_by_site_species['species_pct'] = all_pollen_by_site_species.pcount/all_pollen_by_site_species.site_total*100
all_pollen_by_site_species = all_pollen_by_site_species.drop(['pcount', 'site_total'], axis=1).reset_index()
print all_pollen_by_site_species.head()

Add `varname` and `lct` coulmns from `unique_species`

In [ ]:
#merge in species info
all_pollen_by_site_species = all_pollen_by_site_species.merge(unique_species, on='varcode', how='left')
# sort by site and percent pollen contributed by species
all_pollen_by_site_species = all_pollen_by_site_species.sort_values(by=['sitecode','species_pct'], ascending=False).set_index('sitecode')
# replace None with the string 'not_specified' in the joined in lct column
all_pollen_by_site_species.lct = all_pollen_by_site_species.lct.fillna('not_specified')


In [ ]:
print all_pollen_by_site_species.head()

In [ ]:
pol_accounted_by_lct = all_pollen_by_site_species.reset_index().groupby(['sitecode', 'lct']).sum()
pol_accounted_by_lct = pol_accounted_by_lct.unstack()

In [ ]:
pol_accounted_by_lct

In [ ]:
f, ax = plt.subplots()
pol_accounted_by_lct.plot(kind='bar', stacked=True, ax=ax)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
unexplained_pollen = all_pollen_by_site_species[all_pollen_by_site_species.lct=='not_specified']
for i in unexplained_pollen.index.unique():
    print unexplained_pollen.loc[i].drop('lct', axis=1).head()
    print '\n'

The proportions of pollen not falling into one of the groups represented in the model above is deeped acceptable, i.e. at least 90% of pollen for simulated study sites is attributed to a modelled land cover type.

### REPLACE Consider only most abundant species overall across study sites
- At this point, each species in a list of the identified species with the most pollen accross all sites is matched manually to one of grass, seeder, sprouter, shrub or exclude in an external csv file. 
- This is sub-optimal as the files are edited manually, making it difficult for someone else to track the logic -- we should exploit the readability of Python to make this explicit.
- Instead, propose to use a series of regular expressions to create a map between species and plant functional type
- Target should be to get at least 90% of pollen for each site accounted for in one of the landcover types I propose as part of my model. 
- Since the grouping of species into different functional types effectievly defines a model, it should be set up so a list of regular expressions conditions can be passed to a function to apply them and return the resulting dataframe in the expected format.

In [ ]:
def get_top_species(n):
    #return a dataframe giving top n species 
    # for all cores in epd_data   
    res = pd.DataFrame(columns=['var_', 'varname', 'count', 'sitename'])
    for ssite in epd_data.sitename.unique():
        df = epd_data[epd_data.sitename==ssite]
        df = df.groupby(['var_', 'varname']).agg({'count' : 'sum'}).reset_index()
        df['sitename'] = ssite
        df = df.sort_values(by='count', ascending=False).head(n)
        res = res.append(df)
        res = res.reset_index(drop=True)
        res.var_ = res.var_.astype('int')
        res = res[['sitename', 'var_', 'varname', 'count']]
    return res

In [ ]:
top_species = get_top_species(10)
top_species = top_species.groupby(['var_', 'varname']).agg({'sitename' : 'count'})
top_species = top_species.sort_values('sitename', ascending=False)
top_species.columns = ['sitecount']

In [ ]:
print top_species

Add to this list any variables which are some form of pinus which have not already been identified.

In [ ]:
pinus_species = epd_data[epd_data.varname.str.lower().str.contains('pinus')][['var_', 'varname']].drop_duplicates()
print pinus_species

Drop the Carpinus types (these are actually in the Birch family)

In [ ]:
pinus_species = pinus_species.drop([438, 8], axis=0).set_index('var_')
print pinus_species

Find pinus species not already listed in `top_species`

In [ ]:
pinus_species = pinus_species.join(top_species.reset_index(1)['sitecount'], how='left')
pinus_species = pinus_species[pinus_species.sitecount.isnull()].fillna(0)
pinus_species = pinus_species.reset_index().set_index(['var_','varname'])
print pinus_species

Append to `top_species`

In [ ]:
top_species = top_species.append(pinus_species)

In [ ]:
top_species.to_csv('top_species.csv')

### REPLACE Edit list of species manually outside of notebook

In [ ]:
top_species = pd.read_csv('top_species_edited.csv',
                         index_col='var_')

In [ ]:
top_species

### Prepare top species for plotting

In [ ]:
top_species[(top_species.group<>'exclude') & (top_species.group.notnull())]

In [ ]:
epd_data = epd_data.join(top_species['group'], on='var_')

In [ ]:
epd_data.head()

Get total pollen contributed by each group for each entity and sample

In [ ]:
group_counts = epd_data.groupby(['e_', 'sample_', 'group']).agg({'count' : 'sum'})
epd_data = epd_data.join(group_counts, on=['e_', 'sample_', 'group'], rsuffix='_group_tot')

In [ ]:
group_counts.head()

Create a new dataframe, `top_epd_data` to store only those rows from `epd_data` where the entry corresponds to one of hte groups considered (shrub, sprouter, grass and seeder). First find unique combinations of values relevant for specifying pollen counts at the group level. This means dropping columns used for identifying species.

In [ ]:
top_epd_data = epd_data.drop(['var_', 'count', 'varcode', 'varname', 'pollen_pct'], axis=1)
top_epd_data = epd_data[['e_', 'sample_', 'agebp', 'site_', 'sitename', 'count_sample_tot', 'group']].drop_duplicates()
top_epd_data.head()

Remove rows corresponding to excluded or unspecified groups

In [ ]:
top_epd_data = top_epd_data[(top_epd_data.group<>'exclude') & (top_epd_data.group.notnull())]

In [ ]:
top_epd_data.head(10)

Join in total pollen counts by group from `group_counts` data frame

In [ ]:
top_epd_data = top_epd_data.join(group_counts, on=['e_', 'sample_', 'group'])

In [ ]:
top_epd_data['group_pct'] = top_epd_data['count']/top_epd_data.count_sample_tot*100
top_epd_data.head()

In [ ]:
top_epd_data.columns = ['e_', 'sample_', 'agebp', 'site_', 'sitename', 'count_sample_tot',
                      'group', 'count_group_tot', 'group_pct']

In [ ]:
top_epd_data.head()

Finally pivot dataframe to provide easier access to data for plotting and statistical analyses.

In [ ]:
pol_df = top_epd_data.drop(['e_', 'sample_', 'site_', 'count_sample_tot'], axis=1)
pol_df.columns = ['agebp', 'sitename', 'group', 'group_count', 'group_pct']
pol_df.head()

In [ ]:
pol_df = pol_df.set_index(['sitename', 'agebp', 'group'])

Charco da Candieira contains multiple entries for three samples. At time of writing (4 days before upgrade report is due) I simply don't have time to debug this. 

In [ ]:
pol_df[pol_df.index.duplicated(False)]

For now I'll do a quick and dirty exclusion of all but the first of each of these entries, but will need to address more carefully as a TODO.

In [ ]:
pol_df = pol_df.drop_duplicates(keep='first')

In [ ]:
pol_df = pol_df[~pol_df.index.duplicated(keep='first')]

In [ ]:
pol_df[pol_df.index.duplicated(False)]

In [ ]:
pol_df = pol_df.unstack()
pol_df = pol_df.fillna(0)
pol_df.head()

In [ ]:
pol_df.to_pickle('pol_df.pickle')

### Plot pollen data

In [ ]:
try:
    pol_df
except NameError:
    pol_df = pd.read_pickle('pol_df.pickle')
    print 'pol_df read from file.'

#### For print

In [ ]:
def plot_print_chronology(sitename, earliest, latest, figlabel=None, save=False):
    df = pol_df.loc[sitename, :]['group_pct'] #extract pollen percents for specified site
    df = df[(df.index <= earliest) & (df.index >= latest)] # exclude samples from earlier that specified years before present
    
    def tweak_pct_ticks(axis, pct_vals):
        max_pct = int(round(pct_vals.max()*1.1))
        
        def get_increments(maximum):
            while maximum%4 <> 0:
                maximum += 1
            return [maximum/4 * i for i in range(5)]
        
        increments = get_increments(max_pct)
        axis.set_xlim(0, increments.pop())
        axis.xaxis.set_ticks(increments)
        
    def make_under_line_polygon(xx, yy, e, l):
        line_vertices = np.column_stack((xx, yy))
        leftmost_corners = np.array([[0, e], [0,l]])
        vertices = np.concatenate((line_vertices, leftmost_corners))
        return Polygon(vertices, True)       
    
    pollen_line_colour = '#145D85'
    
    f, axes = plt.subplots(1, len(df.columns), sharey=True)
    for i, group in enumerate(df.columns):
        xx = df[group].values
        yy = df.index.values
        axes[i].plot(xx,yy, color=pollen_line_colour)
        axes[i].set_title(group.title())
        axes[i].set_ylim([latest, earliest])
        tweak_pct_ticks(axes[i], xx)
        
        poly = make_under_line_polygon(xx, yy, earliest, latest)
        p = PatchCollection([poly], alpha=0.4)
        p.set_color(pollen_line_colour)
        axes[i].add_collection(p)
        
        if i == 0:
            axes[i].set_ylabel('yrs BP', fontsize=13)
            if figlabel:
                xticks = axes[i].get_xticks()
                yticks = axes[i].get_yticks()
                xtick_scale = xticks[1]-xticks[0]
                ytick_scale = yticks[1]-yticks[0]

                axes[i].text(-1.15*xtick_scale, latest-0.5*ytick_scale, 
                             figlabel,
                             fontdict = {'weight': 'bold',
                                         'size': 16}
                            )
    
    plt.gca().invert_yaxis()
    plt.subplots_adjust(hspace=0, wspace=0)
    f.text(0.51, 0.02, '% contribution to total pollen sample', ha='center', fontsize=13)
    #plt.suptitle(sitename, y=1.05, fontsize=12)
    
    if save:
        d = os.path.join('plots')
        if not os.path.exists(d):
            os.makedirs(d)

        plt.savefig(os.path.join('plots',
                                 (sitename.replace(' ', '_')+'_'
                                 +str(earliest)+'-'+str(latest)+'.pdf')))

In [ ]:
for s in pol_df.index.get_level_values(0).unique():
    print s
    plot_print_chronology(s, 15000, 0)

Of these, to my eye, San Rafael looks the most interesting (like there's a lot going on). 

On the other hand, what's going on in Navarres at 6000 years ago with sprouters?

#### Interactive

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import gridplot, widgetbox, column# container for bokeh figure objects
from bokeh.models.widgets import Dropdown
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
output_notebook()

In [ ]:
def plot_interactive_chronology(sitename):
    df = pol_df.loc[sitename, :]['group_pct'] #extract pollen percents for specified site
    
    # create a column data source for the plots to share
    source = ColumnDataSource(data=df.reset_index().to_dict('list'))
    
    # container for bokeh figure objects
    plots = [] 
    time_range=None
    
    TOOLS = "ypan,ywheel_zoom"
    
    def get_width(base, factor, plot_num):
        # function to increase width of first plot, since this ends up narrowed
        # due to being the only one with yaxis labels.
        if plot_num > 0:
            return base
        else:
            return int(round(base*(1+factor)))
    
    for i, group in enumerate(df.columns):
        p = figure(tools=TOOLS, plot_width=get_width(150, .25, i), 
                   plot_height=500, y_range=time_range,
                   title=group.title())
        p.line(group, 'agebp', source=source)
        if i == 0:
            p.y_range.flipped = True
            time_range = p.y_range
        else:
            p.yaxis.major_label_text_font_size = '0pt'
                    
        plots.append(p)
   
    p = gridplot([plots])
    t = show(p, notebook_handle=True)
                
    return t

In [ ]:
for s in pol_df.index.get_level_values(0).unique():
    print s

In [ ]:
pol_df.head()

In [ ]:
loc[u'Charco da Candieira', :]['count_group_tot']

Earliest date for Charco da Candieira:

In [ ]:
def print_daterange(sitename):
    df = pol_df.loc[sitename, :]['group_count']
    latest = df.index.min()
    earliest = df.index.max()
    print 'earliest date: {0} yr BP'.format(earliest)
    print 'latest date: {0} yr BP'.format(latest)   

In [ ]:
for s in pol_df.index.get_level_values(0).unique():
    print s
    print_daterange(s)
    print '\n'

In [ ]:
plot_interactive_chronology(u'Algendar')

In [ ]:
pol_df.head()

#### Points of particular interest in time series (discussed in upgrade report)

##### San Rafael 4000 - 8000 yrs BP
Big variation in grasses shrubs and sprouters around the time it is thought agriculture reached Iberia (6500 yrs BP).

In [ ]:
plot_print_chronology(u'San Rafael', 8500, 1000, figlabel='A', save=True)

##### Navarres 6000 - 7000 yrs BP
~ 200 year oscillation in percentages of grass and seeders 6400 - 6800 yrs BP, followed by sudden and sustained increase in sprouters after 6400 yrs BP

In [ ]:
plot_print_chronology(u'Navarrés', 10500, 3000, figlabel='B', save=True)

## 4. Apply the LRA to infer land-cover proportion from pollen abundance

## 5. TODO Output plant functional group time-series for each study site

### TEMP Time-series of proportion of total pollen abundance for each plant functional group
- NOTE at present (May 18) I've not implemented the LRA yet so will output pollen _abundance_ between species, rather than using the LRA's method of correcting for the variance in pollen produced by different species.
- This is to get a preliminary model off the ground and should be corrected for as a priority.

Load pollen chronologies for study sites, and mappings to land cover classes

In [ ]:
if 'epd_data' not in locals():
    import matplotlib 
    import matplotlib.pyplot as plt
    %matplotlib inline
    import pandas as pd
    import numpy as np
    # made in section 2 above
    epd_data = pd.read_pickle('epd_data.pkl')
    
if 'mapped_species' not in locals():
    # made in section 3 above
    mapped_species = pd.read_csv('species_to_landcover_mapping.csv')

In [ ]:
print epd_data.head()
print epd_data[epd_data.pcount>2000]

note the very high counts for san_rafael. Are these realistic?

In [ ]:
print epd_data.shape

In [ ]:
epd_data = epd_data.reset_index().merge(mapped_species.drop('varname', axis=1), on='varcode', how='left')
epd_data = epd_data.dropna()
print epd_data.shape
print epd_data.head()

In [ ]:
pollen_abundance = epd_data.groupby(['sitecode', 'e_', 'agebp', 'lct']).sum().unstack(3)
pollen_abundance = pollen_abundance.fillna(0)
pollen_abundance.loc[:,('pcount', 'total')] = pollen_abundance.sum(axis=1)

In [ ]:
pollen_abundance.head()

Convert abundance to proportion

In [ ]:
for c in pollen_abundance.pcount.columns:
    if c <> 'total':
        pollen_abundance.loc[:,('pprop', c)] = pollen_abundance.loc[:, ('pcount', c)]/pollen_abundance.loc[:, ('pcount', 'total')]

In [ ]:
pollen_abundance.head()

Have a quick look at the data to check it seems reasonable

In [ ]:
fig, ax = plt.subplots()
pollen_abundance.loc[('navarres',  471), 'pprop'].plot(ax=ax)
plt.legend(loc='upper right')

write processed pollen proportion data to disk

In [ ]:
pollen_abundance.to_pickle('pollen_timeseries.pkl')

the above are pollen _proportion_ time series. These can be used as proportions feeding into an NLM. See the `/home/andrew/Dropbox/codes/python/notebooks/modified_random_clusters/implement_modified_random_clusters.html` for details

#### TO move across to MRC notebook
notebook for details of Supposing I start simulating Navarres from 7000 yrs BP, that gives me the following starting proportions:

In [ ]:
# Helper function to find the nearest value to a given value in a numpy array
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [ ]:
nav_dat = pollen_abundance.loc[('navarres',  471), 'pprop']
nav_initial = nav_dat.loc[find_nearest(nav_dat.index.values, 7000)]
print nav_initial
#print find_nearest(nav_dat.index.values, 7000)

## Jan 2019 -- Add time derivatives to timeseries dataframe

In [ ]:
pollen_abundance = pd.read_pickle('pollen_timeseries.pkl')

In [ ]:
pollen_abundance.head()

In [ ]:
pollen_abundance.shape

In [ ]:
pollen_abundance['pprop'].xs('albufera_alcudia', level='sitecode').head()

The next step is to work out how to calculate, for each core, the pollen proportion slope with respect to the agebp index. This can be gathered as a new dataframe with the same MultiIndex as `pollen_abundance['pprop']`. This can then be joined back into `pollen_abundance` as `pollen_abundance['pprop_prime']`. The gradient of this will give `pollen_abundance['pprop_prime_prime']`

In [ ]:
help(pollen_abundance.index)

## Appendices

### Rough working

#### Correlations between variables

Let's look at how the counts of different groups correlate with each other within each study site through time.

In [ ]:
top_epd_data.head()

In [ ]:
df['group_pct']['grass'].loc['Sanabria Marsh'].head()

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(df.loc[u'Navarrés''Sanabria Marsh', :]['count_group_tot'])

Are there clusters we can find in the counts of different pollen? Could investigate using KNN.

In [ ]:
pollen_conts.head()

In [ ]:
pollen_conts['pollen_pct']['mean'].sort_values(ascending=False)

In [ ]:
nav_pine = nav_dat[nav_dat.varname=='Pinus']

In [ ]:
nav_pine['count'].hist()

In [ ]:
nav_pine['pollen_pct'].hist()

In [ ]:
nav_dat[nav_dat.varname=='Concentration spikes'].pollen_pct.plot()

### TODO General theory to look up

Looking at understanding pollen spikes
https://quantpalaeo.wordpress.com/2017/07/28/pollen-spikes/

Calculating deposition rates
http://www.europeanpollendatabase.net/wiki/lib/exe/fetch.php?media=epd_age-depth.pdf

Using litholgy (depth) and and c14 (time) or (equivalently??) `depthcm` and `age` columns from `agebasis` table could be used to calculate sediment deposition rates.

In [ ]:
epd.ssites.append(762)
epd.ssites.append(1260)
epd.ssites.append(76)
epd.ssites.append(560)

In [ ]:
epd.ssites